# **关于倒闭企业数据集的数据分析**

In [1]:
# 导入模块
import pandas as pd

from pyecharts import options as opts
from pyecharts.charts import Map
from pyecharts.charts import Bar
from pyecharts.faker import Faker
from pyecharts.charts import Pie
from pyecharts.charts import WordCloud
from pyecharts.globals import SymbolType

import matplotlib.pyplot as plt
import seaborn as sns
import datetime

from IPython.display import IFrame

import warnings
warnings.filterwarnings("ignore")


## **读取数据**

In [2]:
data = pd.read_csv('com_1.csv')
data.shape

(6272, 16)

In [3]:
data.head()

,bianh,com_name,com_addr,cat,se_cat,com_des,born_data,death_data,live_days,financing,total_money,death_reason,invest_name,ceo_name,ceo_des,ceo_per_des
0,1,麦子金服,上海市,金融,借贷,麦子金服是一家移动互联网金融服务集团，为个人和企业提供包括财富管理、股权投资、投融资咨询等综...,2015/3/1,2019/11/25,1730,B轮,90000.0,政策监管 法律法规风险,海通开元&中缔资本,黄大容,董事长,黄大容，80后，经济学硕士，精通中英德三国语言。连续创业者，23岁创立第一家公司，25岁涉足...
1,1,拓道金服,浙江省,金融,借贷,拓道金服是一家专注于汽车抵押贷款这一细分市场的P2P互联网金融公司。拓道金服通过互联网民间的...,2013/11/1,2019/11/24,2214,A轮,8000.0,政策监管 法律法规风险,蓝山中国资本&帮实资本&宏桥资本,张罗军,创始人,杭州拓道科技有限公司执行董事兼总经理，重庆邮电学院计算机专业，创立杭州仁润科技有限公司，具有...
2,1,云柏科技,广东省,医疗健康,医疗器械及硬件,云柏科技是一家智能健康检测腕表研发商，专注于智能可穿戴设备领域。主要产品包括可穿戴式多功能云...,2014/9/1,2019/11/1,1887,尚未获投,NaN,行业竞争,NaN,NaN,NaN,NaN
3,1,迷你生鲜,福建省,电子商务,生鲜食品,迷你生鲜是一家会员制水果生鲜电商网站，平台产品由基地直供，采购深入全国及东南亚各地区，减少中...,2017/11/1,2019/10/29,727,尚未获投,NaN,现金流断裂 行业竞争,NaN,NaN,NaN,NaN
4,1,一生健康,北京市,医疗健康,寻医诊疗,微爱康一款基于移动互联网的癌症患者互助型垂直社区产品，通过在线社会化问答和类轻博客交流模式，...,2015/1/1,2019/10/17,1750,尚未获投,NaN,政策监管 法律法规风险,NaN,张耀斌,CEO,张耀斌，为一生（北京）健康科技有限公司的创始人。


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6272 entries, 0 to 6271
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   bianh         6272 non-null   int64  
 1   com_name      6272 non-null   object 
 2   com_addr      6272 non-null   object 
 3   cat           6272 non-null   object 
 4   se_cat        6271 non-null   object 
 5   com_des       6271 non-null   object 
 6   born_data     6272 non-null   object 
 7   death_data    6272 non-null   object 
 8   live_days     6272 non-null   int64  
 9   financing     6272 non-null   object 
 10  total_money   805 non-null    float64
 11  death_reason  1419 non-null   object 
 12  invest_name   600 non-null    object 
 13  ceo_name      4839 non-null   object 
 14  ceo_des       4838 non-null   object 
 15  ceo_per_des   4839 non-null   object 
dtypes: float64(1), int64(2), object(13)
memory usage: 784.1+ KB


| 序号 |表头          |含义        |
| -----| -----:      | :----:     |
|0     |bianh        |编号        |
|1     |com_name     |公司名称     |
|2     |com_addr     |公司地址     | 
|3     |cat          |类别         |
|4     |se_cat       |细类         |
|5     |com_des      |公司简述     |
|6     |born_data    |创立日期     |
|7     |death_data   |倒闭日期     |
|8     |live_days    |存活时间（天）|
|9     |financing    |是否有投资    |
|10    |total_money  |总营业金额    |
|11    |death_reason |倒闭原因      |
|12    |invest_name  |投资公司名    |
|13    |ceo_name     |公司拥有者名字|
|14    |ceo_des      |公司拥有者职称|
|15    |ceo_per_des  |公司拥有者简述|

简单分析所有表头，有价值的有公司类别、是否有投资、总营业金额、倒闭原因等

In [5]:
data.isnull().sum()

bianh              0
com_name           0
com_addr           0
cat                0
se_cat             1
com_des            1
born_data          0
death_data         0
live_days          0
financing          0
total_money     5467
death_reason    4853
invest_name     5672
ceo_name        1433
ceo_des         1434
ceo_per_des     1433
dtype: int64

可以看到表中有许多确实值，因此对于投资人、倒闭原因等分析通过词云来进行整合

## **数据分析可视化**

### **倒闭公司的地区分布**

In [6]:
data['com_addr'] = data['com_addr'].apply(lambda x: x.strip())
s = data.groupby('com_addr').size()

c = (
    Map()
    .add("倒闭企业数量", [*s.items()], "china")
    .set_global_opts(
        title_opts=opts.TitleOpts(title="地区分布"),
        visualmap_opts=opts.VisualMapOpts(max_=200),
    )
)


c.render("regional_distribution.html")

IFrame(src="./regional_distribution.html", width=800, height=600)

### **行业排行TOP10**

In [7]:
s = data.groupby('cat').size().sort_values(ascending=False)[:10].to_dict()

c = (
    Bar()
    .add_xaxis(list(s.keys()))
    .add_yaxis("倒闭企业数量", list(s.values()))
    .set_global_opts(title_opts=opts.TitleOpts(title="行业排行TOP10"))
)
c.render("industry_ranking.html")

IFrame(src="./industry_ranking.html", width=1000, height=600)

### **行业细类TOP20**

In [8]:
s = data.groupby('se_cat').size().sort_values(ascending=False)[:20].sort_values(ascending=True).to_dict()

c = (
    Bar()
    .add_xaxis(list(s.keys()))
    .add_yaxis("倒闭企业数量", list(s.values()))
    .reversal_axis()
    .set_series_opts(label_opts=opts.LabelOpts(position="right"))
    .set_global_opts(title_opts=opts.TitleOpts(title="细分领域TOP20"))
)

c.render("industry_refined_ranking.html")

IFrame(src="./industry_refined_ranking.html", width=1000, height=600)

### **年份分布**

In [9]:
data['born_year'] = data['born_data'].apply(lambda x: x[:4])
data['death_year'] = data['death_data'].apply(lambda x: x[:4])
s1 = data.groupby('born_year').size()
s2 = data.groupby('death_year').size()
s1 = pd.DataFrame({'year': s1.index, 'born': s1.values})
s2 = pd.DataFrame({'year': s2.index, 'death': s2.values})
s = pd.merge(s1,s2, on='year', suffixes=['born', 'death'])
s = s[s['year'] > '2008']

c = (
    Bar()
    .add_xaxis( s['year'].to_list())
    .add_yaxis("新生企业数量", s['born'].to_list())
    .add_yaxis("倒闭企业数量", s['death'].to_list())
    .set_global_opts(title_opts=opts.TitleOpts(title="年份分布"))
)

c.render("year_distribution.html")

IFrame(src="./year_distribution.html", width=1000, height=600)

### **企业存活时间**

In [10]:
def live_year(x):
    if x < 365:
        return '不到1年'
    if x < 365 * 2:
        return '1-2年'
    if x < 365 * 3:
        return '2-3年'
    if x < 365 * 4:
        return '3-4年'
    if x < 365 * 5:
        return '4-5年'
    if x < 365 * 10:
        return '5-10年'
    return '10年以上'

s = data.groupby(data['live_days'].apply(lambda x: live_year(x))).size()

print([*s.items()])

c = (
    Pie()
    .add("", [*s.items()])
    .set_global_opts(title_opts=opts.TitleOpts(title="企业存活时长"))
    .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: {c}"))
)

c.render("survival_time.html")

IFrame(src="./survival_time.html", width=800, height=600)

[('1-2年', 644), ('10年以上', 60), ('2-3年', 2242), ('3-4年', 1108), ('4-5年', 954), ('5-10年', 1219), ('不到1年', 45)]


### **投资人词云**

In [11]:
invest = {}
for row in data['invest_name'].values:
    if not pd.isnull(row):
        for name in row.split('&'):
            invest[name] = invest.get(name, 0) + 1
invest = [*invest.items()]
invest.sort(key=lambda x: x[1], reverse=True)
c = (
    WordCloud()
    .add("", invest[:150], word_size_range=[20, 100], shape=SymbolType.DIAMOND)
    .set_global_opts(title_opts=opts.TitleOpts(title="投资人词云"))
)

c.render("investor_word_cloud.html")

IFrame(src="./investor_word_cloud.html", width=800, height=600)

### **倒闭原因词云**

In [12]:
death_reason = {}
for row in data['death_reason'].values:
    if not pd.isnull(row):
        for name in row.split(' '):
            death_reason[name] = death_reason.get(name, 0) + 1
c = (
    WordCloud()
    .add("", [*death_reason.items()], word_size_range=[20, 100], shape=SymbolType.DIAMOND)
    .set_global_opts(title_opts=opts.TitleOpts(title="倒闭原因词云"))
)

c.render("closure_reason.html")

IFrame(src="./closure_reason.html", width=800, height=600)

In [13]:
death_reason = {}
for row in data['death_reason'].values:
    if not pd.isnull(row):
        for name in row.split(' '):
            death_reason[name] = death_reason.get(name, 0) + 1

c = (
    Bar()
    .add_xaxis(list(death_reason.keys()))
    .add_yaxis("因此原因倒闭企业数", list(death_reason.values()))
    .set_global_opts(title_opts=opts.TitleOpts(title="倒闭原因"), xaxis_opts=opts.AxisOpts(name_rotate=60,axislabel_opts={"rotate":45}))
)
c.render("reason.html")

IFrame(src="./reason.html", width=1000, height=600)

In [14]:
data = pd.read_csv('com.csv')
data.head()

,bianh,com_name,com_addr,cat,se_cat,com_des,born_data,death_data,live_days,financing,...,death_reason,invest_name,ceo_name,ceo_des,ceo_per_des,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20
0,1,麦子金服,上海,金融,借贷,麦子金服是一家移动互联网金融服务集团，为个人和企业提供包括财富管理、股权投资、投融资咨询等综...,2015-03-01,2019-11-25,1730,B轮,...,政策监管 法律法规风险,海通开元&中缔资本,黄大容,董事长,黄大容，80后，经济学硕士，精通中英德三国语言。连续创业者，23岁创立第一家公司，25岁涉足...,NaN,NaN,NaN,NaN,NaN
1,1,拓道金服,浙江,金融,借贷,拓道金服是一家专注于汽车抵押贷款这一细分市场的P2P互联网金融公司。拓道金服通过互联网民间的...,2013-11-01,2019-11-24,2214,A轮,...,政策监管 法律法规风险,蓝山中国资本&帮实资本&宏桥资本,张罗军,创始人,杭州拓道科技有限公司执行董事兼总经理，重庆邮电学院计算机专业，创立杭州仁润科技有限公司，具有...,NaN,NaN,NaN,NaN,NaN
2,1,云柏科技,广东,医疗健康,医疗器械及硬件,云柏科技是一家智能健康检测腕表研发商，专注于智能可穿戴设备领域。主要产品包括可穿戴式多功能云...,2014-09-01,2019-11-01,1887,尚未获投,...,行业竞争,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,迷你生鲜,福建,电子商务,生鲜食品,迷你生鲜是一家会员制水果生鲜电商网站，平台产品由基地直供，采购深入全国及东南亚各地区，减少中...,2017-11-01,2019-10-29,727,尚未获投,...,现金流断裂 行业竞争,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,一生健康,北京,医疗健康,寻医诊疗,微爱康一款基于移动互联网的癌症患者互助型垂直社区产品，通过在线社会化问答和类轻博客交流模式，...,2015-01-01,2019-10-17,1750,尚未获投,...,政策监管 法律法规风险,NaN,张耀斌,CEO,张耀斌，为一生（北京）健康科技有限公司的创始人。,NaN,NaN,NaN,NaN,NaN


In [15]:
data.isnull().sum()

bianh              0
com_name           0
com_addr           0
cat                0
se_cat             1
com_des            1
born_data          0
death_data         0
live_days          0
financing          0
total_money     5467
death_reason    4853
invest_name     5672
ceo_name        1433
ceo_des         1434
ceo_per_des     1433
Unnamed: 16     6272
Unnamed: 17     6272
Unnamed: 18     6272
Unnamed: 19     6272
Unnamed: 20     6272
dtype: int64

In [16]:
data.duplicated().sum()

0